In [ ]:
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms as Tf
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


In [ ]:
ds = datasets.MNIST('data', download=True)

In [ ]:
class dataset(Dataset):
    def __init__(self, data, targets):
        super().__init__()
        self.len = len(data)
        self.data = data.view((-1, 28 * 28)).float()
        self.data = (self.data - self.data.mean(axis=-1, keepdim=True)) / self.data.std(axis=-1, keepdim=True)


        self.targets = targets

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

    def __len__(self):
        return self.len

In [ ]:
train_ds , test_ds = (ds.data[:50000], ds.targets[:50000]) , (ds.data[50000:], ds.targets[50000:])
train = dataset(*train_ds)
eval = dataset(*test_ds)

t_dl = DataLoader(train, batch_size=128)
e_dl = DataLoader(eval, batch_size=16)


In [ ]:
class simple(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28*28, 64, bias=True)
        self.l2 = nn.Linear(64, 10, bias=True)
        self.ac1 = nn.ReLU()
        self.ac2 = nn.LogSoftmax(dim=1)
    def forward(self, data):
        y0 = self.ac1(self.l1(data))
        y1 = self.ac2(self.l2(y0))
        return y1


In [ ]:
m = simple()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(m.parameters(), lr=0.001)

In [ ]:
def evaluate(m):
    cnt = 0
    for inp, target in e_dl:
        l = m(inp).argmax(axis=-1)
        cnt += (target == l).sum()
    return cnt / len(e_dl) /e_dl.batch_size

In [ ]:
from tqdm.notebook import tqdm

loss_val = []
for epoch in range(5):
    for inp, target in (t:=tqdm(t_dl)):
        optimizer.zero_grad()
        labels = m(inp)
        loss = criterion(labels, target)
        loss.backward()
        optimizer.step()
        loss_val.append(loss.item())
    print(f'accuracy = {evaluate(m):02.4f}')